In [2]:
from importlib import reload
import audioDatastore.AudioDatastore as myads
reload(myads)
import os.path
from collections import Counter
import random
from python_speech_features import mfcc, delta
import scipy.io.wavfile as wav
from spafe.utils import vis
import matplotlib.pyplot as plt
import misc.vad as vad
from sklearn.mixture import GaussianMixture, _gaussian_mixture
reload(vad)
import numpy as np
from scipy.special import logsumexp
from collections import Counter
import helper_functions as hf
reload(hf)
import pickle

In [3]:
# create ads
# female ads
datasetFolder = r"C:\Users\David\OneDrive\Desktop\matlab\data\SPEECH DATA\FEMALE\MIC"
female_ads = myads.AudioDatastore()
female_ads.populate(datasetFolder,include_sub_folders=True, label_source=True)
Counter(female_ads.labels).values()
# male ads
datasetFolder = r"C:\Users\David\OneDrive\Desktop\matlab\data\SPEECH DATA\MALE\MIC"
male_ads = myads.AudioDatastore()
male_ads.populate(datasetFolder,include_sub_folders=True, label_source=True)
Counter(male_ads.labels).values()
# combine ads
ads = myads.AudioDatastore()
ads.set([datasetFolder], files=female_ads.files + male_ads.files, labels=female_ads.labels+male_ads.labels)

In [4]:
# split ads
developmentLabels = ["M01","M02","M03","M04","M06","M07","M08","M09","F01","F02","F03","F04","F06","F07","F08","F09"]
evaluationLabels = ["M05","M10","F05","F10"]
adsTrain = myads.subset(ads, developmentLabels)
adsEvaluate = myads.subset(ads, evaluationLabels)
numFilesPerSpeakerForEnrollment = 3
adsEnroll, adsDet = myads.split(adsEvaluate, 3)
adsTest, adsDet = myads.split(adsDet, 2)

In [17]:
# test removing indices

files = adsTrain.files[0:10]
files.pop(3)
print(len(files))

9


In [5]:
speedUpExamples = True
if speedUpExamples:
    adsTrain, _ = myads.split(adsTrain, 30)
    adsDet, _ = myads.split(adsDet, 21)

In [6]:
# set sample rate,
fs, _ = wav.read(adsTrain.files[0])

In [27]:
reload = False

try:
    if reload :
        raise Exception('reloading')
    training_features = pickle.load(open('training_features.pickle','rb'))
except:
    zero_indices = []
    training_features = []
    for i in range(len(adsTrain.files)):
        if i%50 == 0:
            print('completed ' + str(i) + ' iterations')
        feat = hf.helper_feature_extraction(adsTrain.files[i])
        training_features.append(feat)
    print('completed training feature fe!')
    pickle.dump(training_features, open('training_features.pickle','wb'))

In [ ]:
# remove [] vectors in training files and pop the file from ads

training_features, adsTrain = myads.clean_zeros(adsTrain, training_features)

In [29]:
import copy

print('training files len: ' + str(len(adsTrain.files)) + ' features len: ' + str(len(training_features)))

copy_features = copy.copy(training_features)
print(len(copy_features))
print(len(training_features))
# copy_ads = adsTrain

zero_indices = []
for i in range(len(copy_features)):
    if len(copy_features[i]) == 0:
        zero_indices.append(i)

for index in sorted(zero_indices, reverse=True):
    del copy_features[index]

for i in range(len(copy_features)):
    if len(copy_features[i]) == 0:
        print('nope')

print(len(copy_features))
print(len(training_features))

training files len: 480 features len: 480
480
480
473
480


In [23]:
len(training_features[70]) == 0

True

In [8]:
# collect normalization factors

import numpy as np

means = []
std = []
for feature in training_features:
    if len(feature) > 0:
        means.append(np.mean(feature, axis=0))
        std.append(np.std(feature, axis=0))

means = np.array(means)
means = np.mean(means, axis=0)

std = np.array(std)
std = np.mean(std, axis=0)

class NormFactor:
  def __init__(self, m, s):
    self.means = m
    self.std = s

normFactors = NormFactor(means, std)

pickle.dump(normFactors, open('normFactors.pickle','wb'))

In [10]:
normalised_features = []

for i in range(len(training_features)):
    if len(training_features[i]) > 0:
        normalised = (training_features[i] - means) / std
        normalised = normalised - np.mean(normalised)
        normalised_features.append(normalised)

In [12]:
# train ubm
numComponents = 32
ubm = GaussianMixture(n_components=numComponents, covariance_type='diag')
train_features_flattened = np.array([item for sublist in normalised_features for item in sublist])
ubm.fit(train_features_flattened)
print('sklearn model ll = ' + str(ubm.score(normalised_features[0])))
pickle.dump(ubm, open('ubm.pickle','wb'))

sklearn model ll = -13.556662227987568


In [ ]:
# load
ubm = pickle.load(open('ubm.pickle', 'rb'))
normFactors = pickle.load(open('normFactors.pickle','rb'))

In [13]:
for i in normalised_features:
    if len(i) == 0:
        print('sad face')

In [15]:
# BW stats

import gmmubm.helper_functions as hfgmm

# do stuff
numComponents = 32
numFeatures = normFactors.means.shape[0]

N = []
F = []

for file in adsTrain.files:
    speaker_feature = hf.helper_feature_extraction(file, normFactors)
    if len(speaker_feature) > 0:
        # BW stats, # TODO make this in to function
        n,f,s,l = hfgmm.helper_expectation(speaker_feature, ubm)
        N.append(n)
        F.append(f)
    else:
        print('skipping train file because len = 0, will this create issues ? hmm')

print('BW stats completed')

skipping train file because len = 0, will this create issues ? hmm
skipping train file because len = 0, will this create issues ? hmm
skipping train file because len = 0, will this create issues ? hmm
skipping train file because len = 0, will this create issues ? hmm
skipping train file because len = 0, will this create issues ? hmm
skipping train file because len = 0, will this create issues ? hmm
skipping train file because len = 0, will this create issues ? hmm
skipping train file because len = 0, will this create issues ? hmm
skipping train file because len = 0, will this create issues ? hmm
skipping train file because len = 0, will this create issues ? hmm
skipping train file because len = 0, will this create issues ? hmm
skipping train file because len = 0, will this create issues ? hmm
skipping train file because len = 0, will this create issues ? hmm
skipping train file because len = 0, will this create issues ? hmm


KeyboardInterrupt: 